# 04b - Datasets UC2 : Étude Épidémiologique (Granularité Accident)

---

## Objectif

Créer des **datasets progressifs** pour l'analyse épidémiologique (UC2) à granularité **accident**.

**Question UC2** : "Quels facteurs augmentent le risque d'accident grave ?"

## Différences UC1 vs UC2

| Aspect | UC1 Secours | UC2 Épidémio |
|--------|-------------|---------------|
| **Objectif** | Prioriser les secours | Comprendre les facteurs de risque |
| **Target** | `grave` (35.4%) | `grave` (35.4%) |
| **Granularité** | Accident | Accident (04b) + Usager (04c) |
| **Contrainte** | Temps réel (features à l'appel) | **Aucune** (analyse rétrospective) |
| **Features** | Exclut post-accident | **Toutes incluses** |
| **Technique** | class_weight='balanced' | class_weight='balanced' / BalancedBagging |

> **Note** : UC1 et UC2 utilisent la même target `grave` (au moins 1 tué ou hospitalisé, 35.4%).
> La différence clé est que UC2 a accès à **toutes les features** car c'est une analyse rétrospective.

## Données

- **Input** : `dataset_features_intelligentes.csv` (208,616 × 49)
- **Output** : 3 datasets progressifs UC2 (V1 contexte → V2 véhicules → V3 complet)
- **Notebook suivant** : `04c_dataset_UC2_usager.ipynb` (granularité usager) → `05b_model_UC2_epidemio.ipynb`

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Configuration Plotly
import plotly.io as pio
pio.templates.default = "plotly_white"

# Charger le dataset avec features intelligentes
df = pd.read_csv('../../../données/dataset_features_intelligentes.csv')
print(f"Dataset chargé: {df.shape}")
print(f"\nColonnes disponibles ({len(df.columns)}):")
print(list(df.columns))

Dataset chargé: (208616, 49)

Colonnes disponibles (49):
['lat', 'long', 'vma', 'nbv', 'heure', 'jour_semaine', 'mois', 'age_moyen', 'nb_usagers', 'nb_vehicules', 'nb_passagers', 'nb_pietons', 'nb_hommes', 'nb_femmes', 'weekend', 'heure_pointe', 'nuit', 'hors_agglo', 'nuit_non_eclairee', 'nuit_eclairee', 'bidirectionnelle', 'haute_vitesse', 'route_autoroute', 'route_nationale', 'route_departementale', 'route_communale', 'route_rapide', 'collision_frontale', 'collision_arriere', 'collision_cote', 'collision_solo', 'obstacle_arbre', 'obstacle_fixe_dur', 'obstacle_pieton', 'has_moto', 'has_velo', 'has_edp', 'has_cyclomoteur', 'has_pieton', 'has_vulnerable', 'has_poids_lourd', 'has_bus', 'has_vehicule_lourd', 'collision_asymetrique', 'heure_danger', 'weekend_nuit', 'nuit_hors_agglo', 'mortel', 'grave']


In [2]:
# Vérification des deux targets
print("="*60)
print("COMPARAISON DES TARGETS")
print("="*60)

for target_name in ['mortel', 'grave']:
    n = df[target_name].sum()
    pct = df[target_name].mean() * 100
    ratio = (1 - df[target_name].mean()) / df[target_name].mean()
    print(f"{target_name.upper()}: {n:,} ({pct:.1f}%) - Ratio 1:{ratio:.1f}")

print(f"\n→ UC2 utilise GRAVE : déséquilibre modéré (1:1.8) → EasyEnsemble")

COMPARAISON DES TARGETS
MORTEL: 11,843 (5.7%) - Ratio 1:16.6
GRAVE: 73,796 (35.4%) - Ratio 1:1.8

→ UC2 utilise GRAVE : déséquilibre modéré (1:1.8) → EasyEnsemble


### Observation : Target UC2

**GRAVE (35.4%)** est la même target que UC1 :
- Ratio 1:1.8 → class_weight='balanced' ou BalancedBagging suffisant
- 73,796 exemples positifs pour l'apprentissage
- La différence avec UC1 n'est pas la target mais les **features disponibles** (toutes pour UC2, temps réel seulement pour UC1)

---
## 1. Définition des groupes de features

Pour UC2 (épidémio), on a accès à **toutes les features** car c'est une analyse rétrospective.

### Principe de progression

| Version | Scénario | Information |
|---------|----------|-------------|
| V1 | Contexte accident | Localisation, temporel, route |
| V2 | + Véhicules | Types de véhicules impliqués |
| V3 | + Post-accident | Obstacles, type de collision (COMPLET) |

In [3]:
# === FEATURES UC2 (TOUTES DISPONIBLES) ===

# Niveau 1 : Contexte accident (localisation + temporel + route)
features_v1 = [
    # Localisation
    'lat', 'long',
    # Temporel
    'heure', 'jour_semaine', 'mois',
    'weekend', 'nuit', 'heure_pointe', 'heure_danger',
    # Éclairage
    'nuit_non_eclairee', 'nuit_eclairee',
    # Route
    'vma', 'nbv',
    'hors_agglo', 'bidirectionnelle', 'haute_vitesse',
    'route_autoroute', 'route_nationale', 'route_departementale', 
    'route_communale', 'route_rapide',
    # Combinées
    'nuit_hors_agglo', 'weekend_nuit',
]

# Niveau 2 : + Véhicules impliqués
features_v2 = features_v1 + [
    # Présence de véhicules
    'has_moto', 'has_velo', 'has_edp', 'has_cyclomoteur', 'has_pieton',
    'has_vulnerable',
    'has_poids_lourd', 'has_bus', 'has_vehicule_lourd',
    'collision_asymetrique',
    # Comptages
    'nb_vehicules', 'nb_usagers',
    'nb_pietons', 'nb_hommes', 'nb_femmes',
    'age_moyen',
]

# Niveau 3 : + Post-accident (COMPLET)
# Ces features ne sont pas disponibles en temps réel mais très prédictives
features_v3 = features_v2 + [
    # Type de collision
    'collision_frontale', 'collision_arriere', 'collision_cote', 'collision_solo',
    # Obstacles (INFO POST-ACCIDENT - très prédictif)
    'obstacle_arbre', 'obstacle_fixe_dur', 'obstacle_pieton',
]

# Target
target = 'grave'

print("Groupes de features UC2 définis:")
print("="*60)
print(f"  V1 (contexte accident)    : {len(features_v1):>2} features")
print(f"  V2 (+ véhicules)          : {len(features_v2):>2} features")
print(f"  V3 (+ post-accident)      : {len(features_v3):>2} features (COMPLET)")
print(f"\nTarget: {target.upper()}")

Groupes de features UC2 définis:
  V1 (contexte accident)    : 23 features
  V2 (+ véhicules)          : 39 features
  V3 (+ post-accident)      : 46 features (COMPLET)

Target: GRAVE


In [4]:
# Vérifier que toutes les features existent
print("Vérification des features:")
print("="*60)

all_features = set(features_v3)
missing = [f for f in all_features if f not in df.columns]

if missing:
    print(f"⚠️ Features manquantes: {missing}")
else:
    print("✓ Toutes les features sont présentes dans le dataset.")

# Afficher les features par niveau
print("\nDétail par niveau:")
print("-"*60)
print(f"V1: {features_v1}")
print(f"\nV2 ajout: {[f for f in features_v2 if f not in features_v1]}")
print(f"\nV3 ajout: {[f for f in features_v3 if f not in features_v2]}")

Vérification des features:
✓ Toutes les features sont présentes dans le dataset.

Détail par niveau:
------------------------------------------------------------
V1: ['lat', 'long', 'heure', 'jour_semaine', 'mois', 'weekend', 'nuit', 'heure_pointe', 'heure_danger', 'nuit_non_eclairee', 'nuit_eclairee', 'vma', 'nbv', 'hors_agglo', 'bidirectionnelle', 'haute_vitesse', 'route_autoroute', 'route_nationale', 'route_departementale', 'route_communale', 'route_rapide', 'nuit_hors_agglo', 'weekend_nuit']

V2 ajout: ['has_moto', 'has_velo', 'has_edp', 'has_cyclomoteur', 'has_pieton', 'has_vulnerable', 'has_poids_lourd', 'has_bus', 'has_vehicule_lourd', 'collision_asymetrique', 'nb_vehicules', 'nb_usagers', 'nb_pietons', 'nb_hommes', 'nb_femmes', 'age_moyen']

V3 ajout: ['collision_frontale', 'collision_arriere', 'collision_cote', 'collision_solo', 'obstacle_arbre', 'obstacle_fixe_dur', 'obstacle_pieton']


### Observation : Structure des features UC2

**Différences clés avec UC1** :
- V3 inclut `obstacle_arbre`, `obstacle_fixe_dur` (exclues en UC1)
- V2 inclut `nb_usagers`, `age_moyen` (pas connus à l'appel en UC1)
- **Toutes les features** sont utilisables car analyse rétrospective

---
## 2. Création des datasets

In [5]:
# Définir les versions
datasets = {
    'v1_contexte': features_v1,
    'v2_vehicules': features_v2,
    'v3_complet': features_v3,
}

print("="*70)
print("CRÉATION DES DATASETS UC2")
print("="*70)

created_datasets = {}

for name, features in datasets.items():
    # Créer le dataset
    df_version = df[features + [target]].copy()
    
    # Sauvegarder
    filename = f'../../../données/UC2_{name}.csv'
    df_version.to_csv(filename, index=False)
    
    # Stocker pour analyse
    created_datasets[name] = df_version
    
    # Stats
    n_grave = df_version[target].sum()
    pct_grave = df_version[target].mean() * 100
    
    print(f"\n{name}:")
    print(f"  Shape: {df_version.shape}")
    print(f"  Features: {len(features)}")
    print(f"  Target GRAVE: {n_grave:,} ({pct_grave:.1f}%)")
    print(f"  Fichier: données/UC2_{name}.csv")

CRÉATION DES DATASETS UC2

v1_contexte:
  Shape: (208616, 24)
  Features: 23
  Target GRAVE: 73,796 (35.4%)
  Fichier: données/UC2_v1_contexte.csv

v2_vehicules:
  Shape: (208616, 40)
  Features: 39
  Target GRAVE: 73,796 (35.4%)
  Fichier: données/UC2_v2_vehicules.csv

v3_complet:
  Shape: (208616, 47)
  Features: 46
  Target GRAVE: 73,796 (35.4%)
  Fichier: données/UC2_v3_complet.csv


### Observation : Datasets UC2 créés

**Cohérence vérifiée** :
- Tous les datasets ont **208,616 lignes** (identique à UC1)
- Target GRAVE = **73,796** accidents (35.4%)
- Progression : 24 → 40 → 47 features

---
## 3. Analyse des corrélations par version

In [6]:
print("="*70)
print("TOP 10 CORRÉLATIONS AVEC GRAVE PAR VERSION")
print("="*70)

all_correlations = {}

for name, features in datasets.items():
    print(f"\n--- {name.upper()} ({len(features)} features) ---")
    correlations = []
    for feat in features:
        corr = df[feat].corr(df[target])
        correlations.append((feat, corr))
    
    # Trier par corrélation absolue
    correlations.sort(key=lambda x: abs(x[1]), reverse=True)
    all_correlations[name] = correlations
    
    # Top 10
    for i, (feat, corr) in enumerate(correlations[:10], 1):
        print(f"  {i:>2}. {feat:<25} {corr:>+.3f}")

TOP 10 CORRÉLATIONS AVEC GRAVE PAR VERSION

--- V1_CONTEXTE (23 features) ---
   1. route_departementale      +0.259
   2. hors_agglo                +0.257
   3. route_communale           -0.208
   4. bidirectionnelle          +0.203
   5. vma                       +0.188
   6. lat                       -0.168
   7. nuit_non_eclairee         +0.120
   8. nuit_hors_agglo           +0.118
   9. nbv                       -0.094
  10. nuit_eclairee             -0.087

--- V2_VEHICULES (39 features) ---
   1. route_departementale      +0.259
   2. hors_agglo                +0.257
   3. route_communale           -0.208
   4. bidirectionnelle          +0.203
   5. vma                       +0.188
   6. lat                       -0.168
   7. nb_vehicules              -0.129
   8. nuit_non_eclairee         +0.120
   9. nuit_hors_agglo           +0.118
  10. age_moyen                 +0.114

--- V3_COMPLET (46 features) ---
   1. route_departementale      +0.259
   2. hors_agglo                +

In [7]:
# Visualisation : Evolution des corrélations max par version
versions = list(datasets.keys())
max_corrs = [max(abs(c[1]) for c in all_correlations[v]) for v in versions]
n_features = [len(datasets[v]) for v in versions]

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Bar(x=versions, y=n_features, name="Nb features", marker_color='lightgreen'),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=versions, y=max_corrs, name="Corr max (abs)", 
               mode='lines+markers', marker=dict(size=12), line=dict(color='orange', width=3)),
    secondary_y=True,
)

fig.update_layout(
    title="Evolution des features et corrélation max par version UC2",
    xaxis_title="Version",
    height=400
)
fig.update_yaxes(title_text="Nombre de features", secondary_y=False)
fig.update_yaxes(title_text="Corrélation max (abs)", secondary_y=True)

fig.show()

### Observation : Corrélations avec GRAVE

**Top features pour GRAVE** :

| Rang | Feature | Corrélation |
|------|---------|------------|
| 1 | `route_departementale` | +0.259 |
| 2 | `hors_agglo` | +0.257 |
| 3 | `route_communale` | -0.208 |
| 4 | `bidirectionnelle` | +0.203 |

**Conclusion** : Le **type de route** est le facteur le plus prédictif de la gravité au niveau accident. Les routes départementales, hors agglomération et bidirectionnelles concentrent le risque.

---
## 4. Comparaison UC1 vs UC2

In [8]:
# Comparer les corrélations pour les features communes
print("="*70)
print("COMPARAISON DES CORRÉLATIONS : MORTEL vs GRAVE")
print("="*70)
print(f"{'Feature':<25} {'MORTEL':>10} {'GRAVE':>10} {'Ratio G/M':>10}")
print("-"*55)

comparison_data = []
for feat in features_v3:
    corr_mortel = df[feat].corr(df['mortel'])
    corr_grave = df[feat].corr(df['grave'])
    
    # Ratio (éviter division par zéro)
    if abs(corr_mortel) > 0.01:
        ratio = abs(corr_grave) / abs(corr_mortel)
    else:
        ratio = float('inf') if abs(corr_grave) > 0.01 else 1.0
    
    comparison_data.append({
        'feature': feat,
        'corr_mortel': corr_mortel,
        'corr_grave': corr_grave,
        'ratio': ratio
    })

# Trier par ratio décroissant
comparison_data.sort(key=lambda x: x['ratio'] if x['ratio'] != float('inf') else 999, reverse=True)

# Afficher top 15
for item in comparison_data[:15]:
    ratio_str = f"{item['ratio']:.1f}x" if item['ratio'] != float('inf') else "∞"
    print(f"{item['feature']:<25} {item['corr_mortel']:>+10.3f} {item['corr_grave']:>+10.3f} {ratio_str:>10}")

COMPARAISON DES CORRÉLATIONS : MORTEL vs GRAVE
Feature                       MORTEL      GRAVE  Ratio G/M
-------------------------------------------------------
long                          +0.002     +0.024          ∞
route_rapide                  -0.002     -0.071          ∞
has_moto                      -0.008     +0.062          ∞
has_pieton                    -0.006     -0.013          ∞
has_bus                       +0.000     -0.021          ∞
obstacle_pieton               -0.006     -0.013          ∞
route_autoroute               -0.015     -0.083       5.7x
nbv                           -0.018     -0.094       5.2x
nb_usagers                    -0.010     -0.036       3.6x
lat                           -0.049     -0.168       3.4x
bidirectionnelle              +0.072     +0.203       2.8x
collision_arriere             -0.050     -0.118       2.4x
collision_solo                +0.039     +0.091       2.4x
route_departementale          +0.112     +0.259       2.3x
weekend     

In [9]:
# Visualisation : Scatter plot MORTEL vs GRAVE
df_comp = pd.DataFrame(comparison_data)
df_comp = df_comp[df_comp['ratio'] != float('inf')]  # Exclure infini

fig = px.scatter(
    df_comp,
    x='corr_mortel',
    y='corr_grave',
    text='feature',
    title='Corrélations MORTEL vs GRAVE pour chaque feature',
    labels={'corr_mortel': 'Corrélation avec MORTEL', 'corr_grave': 'Corrélation avec GRAVE'}
)

# Ligne diagonale (y=x)
fig.add_trace(go.Scatter(
    x=[-0.3, 0.3], y=[-0.3, 0.3],
    mode='lines', line=dict(dash='dash', color='gray'),
    name='y=x (même importance)'
))

fig.update_traces(textposition='top center', marker=dict(size=10))
fig.update_layout(height=600, showlegend=True)
fig.show()

### Observation : Différences UC1 vs UC2

**Features plus importantes pour GRAVE que MORTEL** :
- `bidirectionnelle` : 2.8x plus important pour GRAVE
- `route_departementale` : 2.3x plus important pour GRAVE
- `collision_arriere` : 2.4x plus important pour GRAVE

**Features plus importantes pour MORTEL que GRAVE** :
- `has_poids_lourd` : 1.9x plus important pour MORTEL
- `collision_asymetrique` : 1.7x plus important pour MORTEL
- `haute_vitesse` : 5.4x plus important pour MORTEL

**Implication** : Les modèles UC1 et UC2 apprendront des patterns différents.

---
## 5. Analyse des taux de gravité par feature

In [10]:
print("="*85)
print("TAUX DE GRAVITÉ PAR FEATURE")
print("="*85)
print(f"{'Feature':<30} {'N':<10} {'% Grave':<10} {'Ratio':>10}")
print("-"*60)

# Taux de base
base_grave = df['grave'].mean() * 100
print(f"{'BASELINE':<30} {len(df):<10,} {base_grave:<10.1f} {'1.0x':>10}")
print("-"*60)

# Calculer pour chaque feature
grave_rates = []
for feat in features_v3:
    subset = df[df[feat] == 1]
    n = len(subset)
    if n > 100:  # Ignorer les features trop rares
        grave_rate = subset['grave'].mean() * 100
        ratio = grave_rate / base_grave
        grave_rates.append({
            'feature': feat,
            'n': n,
            'grave_rate': grave_rate,
            'ratio': ratio
        })

# Trier par ratio décroissant
grave_rates.sort(key=lambda x: x['ratio'], reverse=True)

for item in grave_rates[:15]:
    print(f"{item['feature']:<30} {item['n']:<10,} {item['grave_rate']:<10.1f} {item['ratio']:.2f}x")

TAUX DE GRAVITÉ PAR FEATURE
Feature                        N          % Grave         Ratio
------------------------------------------------------------
BASELINE                       208,616    35.4             1.0x
------------------------------------------------------------
obstacle_arbre                 5,517      75.1       2.12x
obstacle_fixe_dur              15,650     68.8       1.94x
nuit_hors_agglo                18,201     53.6       1.51x
collision_frontale             21,985     51.9       1.47x
nuit_non_eclairee              23,133     51.6       1.46x
hors_agglo                     76,269     51.6       1.46x
route_departementale           79,679     51.1       1.45x
nb_usagers                     34,720     50.7       1.43x
collision_solo                 20,439     48.5       1.37x
collision_asymetrique          4,004      47.2       1.33x
has_poids_lourd                9,742      45.8       1.29x
nb_vehicules                   79,083     44.3       1.25x
heure_danger  

In [11]:
# Visualisation : Top features par taux de gravité
df_rates = pd.DataFrame(grave_rates)
df_rates = df_rates.sort_values('ratio', ascending=True).tail(15)

fig = px.bar(
    df_rates,
    x='ratio',
    y='feature',
    orientation='h',
    color='grave_rate',
    color_continuous_scale='Oranges',
    title='Top 15 features par ratio de gravité (vs baseline 35.4%)',
    labels={'ratio': 'Ratio vs baseline', 'feature': 'Feature', 'grave_rate': '% Gravité'}
)

fig.add_vline(x=1, line_dash="dash", line_color="gray", annotation_text="Baseline")
fig.update_layout(height=500, showlegend=False)
fig.show()

### Observation : Taux de gravité par feature

**Top 5 features avec le plus haut taux de GRAVE** :

| Feature | Taux GRAVE | Ratio |
|---------|------------|-------|
| `obstacle_arbre` | ~75% | **2.1x** |
| `obstacle_fixe_dur` | ~69% | **1.9x** |
| `nuit_hors_agglo` | ~54% | **1.5x** |
| `collision_frontale` | ~52% | **1.5x** |
| `hors_agglo` | ~52% | **1.5x** |

**Observation** : Les mêmes features sont dangereuses pour MORTEL et GRAVE, mais avec des intensités différentes.

---
## 6. Vérification des valeurs manquantes

In [12]:
print("="*70)
print("VALEURS MANQUANTES PAR DATASET")
print("="*70)

for name, df_v in created_datasets.items():
    missing = df_v.isnull().sum()
    total_missing = missing.sum()
    
    print(f"\n{name}: {total_missing} valeurs manquantes")
    
    if total_missing > 0:
        missing_cols = missing[missing > 0]
        for col, n in missing_cols.items():
            pct = n / len(df_v) * 100
            print(f"  - {col}: {n:,} ({pct:.1f}%)")
    else:
        print("  ✓ Aucune valeur manquante")

VALEURS MANQUANTES PAR DATASET

v1_contexte: 0 valeurs manquantes
  ✓ Aucune valeur manquante

v2_vehicules: 0 valeurs manquantes
  ✓ Aucune valeur manquante

v3_complet: 0 valeurs manquantes
  ✓ Aucune valeur manquante


### Observation : Qualité des données

**Excellent** : Aucune valeur manquante dans les 3 datasets UC2.

Les datasets sont prêts pour la modélisation.

---
## 7. Résumé et recommandations

In [13]:
print("="*70)
print("RÉSUMÉ UC2 - ÉTUDE ÉPIDÉMIOLOGIQUE")
print("="*70)

print(f"\nTarget: GRAVE ({df['grave'].sum():,} cas, {df['grave'].mean()*100:.1f}%)")
ratio = (1 - df['grave'].mean()) / df['grave'].mean()
print(f"Ratio classe: 1:{ratio:.1f} → EasyEnsemble recommandé")

print(f"\nDatasets créés:")
print(f"  - UC2_v1_contexte.csv   : {len(features_v1):>2} features (localisation + temporel + route)")
print(f"  - UC2_v2_vehicules.csv  : {len(features_v2):>2} features (+ véhicules + usagers)")
print(f"  - UC2_v3_complet.csv    : {len(features_v3):>2} features (+ obstacles + collision)")

print(f"\n→ Prochaine étape: Phase 4 - Modélisation avec EasyEnsemble")

RÉSUMÉ UC2 - ÉTUDE ÉPIDÉMIOLOGIQUE

Target: GRAVE (73,796 cas, 35.4%)
Ratio classe: 1:1.8 → EasyEnsemble recommandé

Datasets créés:
  - UC2_v1_contexte.csv   : 23 features (localisation + temporel + route)
  - UC2_v2_vehicules.csv  : 39 features (+ véhicules + usagers)
  - UC2_v3_complet.csv    : 46 features (+ obstacles + collision)

→ Prochaine étape: Phase 4 - Modélisation avec EasyEnsemble


---
## Conclusions

### Datasets créés

| Dataset | Features | Contenu |
|---------|----------|---------|
| `UC2_v1_contexte.csv` | 23 | Localisation + temporel + route |
| `UC2_v2_vehicules.csv` | 39 | + types véhicules + comptages + usagers |
| `UC2_v3_complet.csv` | 46 | + obstacles + types de collision (COMPLET) |

### Observations clés

1. **Top facteurs de risque** : `route_departementale` (+0.259), `hors_agglo` (+0.257), `obstacle_fixe_dur` (+0.199)
2. **Obstacle arbre** = 75% de gravité (2.1× baseline) → facteur environnemental majeur
3. **Progression V1→V3** : les features post-accident (obstacles, collisions) apportent du signal supplémentaire (+0.199 pour `obstacle_fixe_dur`)

### Différence avec UC1

UC2 utilise **toutes les features** (analyse rétrospective), incluant les données post-accident (`obstacle_arbre`, `obstacle_fixe_dur`, `collision_frontale`) qui sont exclues de UC1 (pas disponibles au moment de l'appel d'urgence).

### Prochaine étape

→ **04c_dataset_UC2_usager.ipynb** : Datasets à granularité usager (features individuelles : âge, sexe, ceinture)
→ **05b_model_UC2_epidemio.ipynb** : Modélisation avec BalancedBagging

---
## Notes techniques

### Comparaison UC1 vs UC2

| Aspect | UC1 (Secours) | UC2 (Épidémio) |
|--------|---------------|----------------|
| **Target** | `grave` (35.4%) | `grave` (35.4%) |
| **Ratio classes** | 1:1.8 | 1:1.8 |
| **Technique** | class_weight='balanced' | class_weight / BalancedBagging |
| **Features** | Temps réel seulement | **Toutes** |
| **Granularité** | Accident | Accident (04b) + Usager (04c) |
| **Split** | Temporel (2021-2023 / 2024) | Random stratifié |
| **Objectif** | Prioriser secours | **Comprendre facteurs** |

### Pistes pour la prévention (insights épidémio)

- **Routes départementales** : Infrastructure prioritaire (+0.259 corrélation)
- **Bidirectionnelles** : Séparateurs centraux ? (+0.203 corrélation)
- **Obstacles fixes** : Élagage des arbres, protections (+0.199 corrélation)
- **Nuit hors agglo** : Éclairage public (+0.118 corrélation)

### Fichiers générés

| Fichier | Shape | Target |
|---------|-------|--------|
| `UC2_v1_contexte.csv` | 208,616 × 24 | `grave` (35.4%) |
| `UC2_v2_vehicules.csv` | 208,616 × 40 | `grave` (35.4%) |
| `UC2_v3_complet.csv` | 208,616 × 47 | `grave` (35.4%) |

### Prochaine étape

→ **04c_dataset_UC2_usager.ipynb** : Granularité usager (477k lignes, 12 features individuelles)
→ **05b_model_UC2_epidemio.ipynb** : Modélisation + facteurs de risque